# HW3 Image Classification
## We strongly recommend that you run with Kaggle for this homework


# Get Data
Notes: if the links are dead, you can download the data directly from Kaggle and upload it to the workspace, or you can use the Kaggle API to directly download the data into colab.


In [1]:
# ! wget https://www.dropbox.com/s/6l2vcvxl54b0b6w/food11.zip

In [2]:
# ! unzip food11.zip

# Training

In [3]:
_exp_name = "boss"

In [4]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

/home/strive236/anaconda3/envs/ml2022spring/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

## **Transforms**
Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [6]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    transforms.RandomChoice([
        transforms.RandomRotation((-30,30)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ColorJitter(brightness=(0.5,1.5), contrast=(0.5, 1.5), saturation=(0.5,1.5), hue=(-0.25, 0.25)),
        transforms.RandomInvert(p=0.5),
        transforms.RandomAffine(degrees=(-30,30), translate=(0.1, 0.1), scale=(0.8, 1.2), shear=(-30, 30)),
        transforms.Grayscale(num_output_channels=3),
    ]),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])


## **Datasets**
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [7]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        if files != None:
            self.files = files
        else:
            self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [8]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride, 0),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out += identity
        out = self.relu(out)
        
        return out

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        # 初始卷积层
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, stride=2, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.block1 = BasicBlock(64, 64, 1) # [64, 64, 64]

        self.block2 = BasicBlock(64, 64) # [64, 64, 64]

        self.block3 = BasicBlock(64, 64) # [64, 64, 64]

        self.block4 = BasicBlock(64, 128, 2) # [128, 32, 32]

        self.block5 = BasicBlock(128, 128) # [128, 32, 32]

        self.block6 = BasicBlock(128, 128) # [128, 32, 32]

        self.block7 = BasicBlock(128, 128) # [128, 32, 32]

        self.block8 = BasicBlock(128, 256, 2) # [256, 16, 16]

        self.block9 = BasicBlock(256, 256) # [256, 16, 16]

        self.block10 = BasicBlock(256, 256) # [256, 16, 16]

        self.block11 = BasicBlock(256, 256) # [256, 16, 16]

        self.block12 = BasicBlock(256, 256) # [256, 16, 16]

        self.block13 = BasicBlock(256, 256) # [256, 16, 16]

        self.block14 = BasicBlock(256, 512, 2) # [512, 8, 8]

        self.block15 = BasicBlock(512, 512) # [512, 8, 8]

        self.block16 = BasicBlock(512, 512) # [512, 8, 8]

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512, 11)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)
        out = self.block6(out)
        out = self.block7(out)
        out = self.block8(out)
        out = self.block9(out)
        out = self.block10(out)
        out = self.block11(out)
        out = self.block12(out)
        out = self.block13(out)
        out = self.block14(out)
        out = self.block15(out)
        out = self.block16(out)

        out = self.avgpool(out)

        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [9]:
batch_size = 64
_dataset_dir = "./food11"

# N-Fold+Ensemble

In [10]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# The number of training epochs and patience.
n_epochs = 200
patience = 50 # If no improvement in 'patience' epochs, early stop

import numpy as np
from sklearn.model_selection import KFold

from torch.utils.tensorboard import SummaryWriter
import datetime

# 初始化5折交叉验证
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

# 加载完整训练集（用于交叉验证）
train_set = FoodDataset(os.path.join(_dataset_dir, "training"), tfm=train_tfm)
valid_set = FoodDataset(os.path.join(_dataset_dir, "validation"), tfm=train_tfm)

# 合并数据集
combined_files = train_set.files + valid_set.files
full_dataset = FoodDataset(path="", tfm=train_tfm, files=combined_files)

oof_preds = np.zeros(len(full_dataset))  # 存储OOF预测结果
oof_labels = np.zeros(len(full_dataset)) # 存储真实标签

# 存储所有基模型（用于后续集成）
base_models = [] 

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
log_dir = f"runs/food_classification_{timestamp}"
writer = SummaryWriter()

for fold, (train_idx, val_idx) in enumerate(kf.split(train_set)):
    print(f"\n====== Fold {fold+1}/{n_folds} ======")
    
    # 划分训练集和验证子集
    train_subset = Subset(train_set, train_idx)
    val_subset = Subset(train_set, val_idx)
    
    # DataLoader
    train_loader = DataLoader(
        train_subset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=12,
        pin_memory=True,
        persistent_workers=True
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8,
        pin_memory=True,
        persistent_workers=True
    )

    # 每折独立初始化模型和优化器
    model = Classifier().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()
    
    # 早停相关变量（每折独立）
    fold_best_acc = 0
    stale = 0

    # 训练循环（保持原有逻辑）
    for epoch in range(n_epochs):
        # ---------- Training ----------
        model.train()
        train_loss, train_accs = [], []
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            imgs, labels = batch
            imgs, labels = imgs.to(device), labels.to(device)
            
            logits = model(imgs)
            loss = criterion(logits, labels)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            optimizer.step()
            
            acc = (logits.argmax(dim=-1) == labels).float().mean()
            train_loss.append(loss.item())
            train_accs.append(acc.item())
        
        # 打印训练信息
        avg_loss = np.mean(train_loss)
        avg_acc = np.mean(train_accs)

        # 写入TensorBoard
        writer.add_scalar(f'Fold_{fold}/Train/Loss', avg_loss, epoch)
        writer.add_scalar(f'Fold_{fold}/Train/Accuracy', avg_acc, epoch)

        print(f"[ Train | {epoch+1:03d}/{n_epochs:03d} ] loss = {avg_loss:.5f}, acc = {avg_acc:.5f}")

        # ---------- Validation ----------
        model.eval()
        val_loss, val_accs, val_preds = [], [], []
        val_labels = []  # 累积所有验证批次的标签

        for batch in tqdm(val_loader, desc="Validating"):
            imgs, labels = batch
            imgs = imgs.to(device)
            labels_np = labels.numpy()
            val_labels.extend(labels_np)  # 累积标签
            
            with torch.no_grad():
                logits = model(imgs)
                preds = logits.argmax(dim=-1).cpu().numpy()
            
            loss = criterion(logits, labels.to(device))
            val_loss.append(loss.item())
            val_accs.append((preds == labels_np).mean())
            val_preds.extend(preds)

        # 记录OOF预测和标签
        oof_preds[val_idx] = np.array(val_preds)
        oof_labels[val_idx] = np.array(val_labels) 

        # 打印验证信息
        avg_val_loss = np.mean(val_loss)
        avg_val_acc = np.mean(val_accs)

        # 写入TensorBoard
        writer.add_scalar(f'Fold_{fold}/Val/Loss', avg_val_loss, epoch)
        writer.add_scalar(f'Fold_{fold}/Val/Accuracy', avg_val_acc, epoch)

        print(f"[ Valid | {epoch+1:03d}/{n_epochs:03d} ] loss = {avg_val_loss:.5f}, acc = {avg_val_acc:.5f}")

        # 早停逻辑（每折独立）
        if avg_val_acc > fold_best_acc:
            print(f"Fold {fold} best model at epoch {epoch}")
            torch.save(model.state_dict(), f"fold{fold}_best.ckpt")
            fold_best_acc = avg_val_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"Early stopping at epoch {epoch}")
                break
    
    # 保存当前折的模型
    base_models.append(model)

# 关闭TensorBoard的writer
writer.close()

# ---------- 后处理 ----------
# 计算OOF准确率
oof_acc = (oof_preds == oof_labels).mean()
print(f"\n[OOF Accuracy] {oof_acc:.4f}")

One ./food11/training sample ./food11/training/0_0.jpg
One ./food11/validation sample ./food11/validation/0_0.jpg
One  sample ./food11/training/0_0.jpg

====== Fold 1/5 ======


Epoch 1: 100%|██████████| 124/124 [00:26<00:00,  4.64it/s]


[ Train | 001/200 ] loss = 2.23542, acc = 0.21184


Validating: 100%|██████████| 31/31 [00:03<00:00, 10.14it/s]


[ Valid | 001/200 ] loss = 2.19749, acc = 0.21223
Fold 0 best model at epoch 0


...

[ Valid | 199/200 ] loss = 1.18050, acc = 0.73385


Epoch 200: 100%|██████████| 124/124 [00:24<00:00,  5.04it/s]


[ Train | 200/200 ] loss = 0.08774, acc = 0.97101


Validating: 100%|██████████| 31/31 [00:02<00:00, 11.99it/s]

[ Valid | 200/200 ] loss = 1.24031, acc = 0.70754

[OOF Accuracy] 0.7787


# Testing and generate prediction CSV

In [11]:
test_set = FoodDataset(os.path.join(_dataset_dir,"test"), tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)

One ./food11/test sample ./food11/test/0001.jpg


# Ensemble

In [ ]:
# 集成预测（软投票法）
all_preds = []
for model in base_models:
    model.eval()
    fold_preds = []
    for data, _ in test_loader:  # 保持和原test_loader一致
        with torch.no_grad():
            logits = model(data.to(device))
            # 保存每个模型的原始logits（概率），而不是直接argmax
            fold_preds.append(logits.cpu().numpy())
    # 合并当前模型的所有batch预测结果
    fold_preds = np.concatenate(fold_preds, axis=0)
    all_preds.append(fold_preds)

# 软投票：平均所有模型的logits后取argmax
all_preds = np.stack(all_preds)  # shape: (n_models, n_samples, n_classes)
prediction = all_preds.mean(axis=0).argmax(axis=1)  # shape: (n_samples,)

In [17]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)

# Q1. Augmentation Implementation
## Implement augmentation by finishing train_tfm in the code with image size of your choice. 
## Directly copy the following block and paste it on GradeScope after you finish the code
### Your train_tfm must be capable of producing 5+ different results when given an identical image multiple times.
### Your  train_tfm in the report can be different from train_tfm in your training code.


In [14]:
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You need to add some transforms here.
    transforms.ToTensor(),
])

# Q2. Residual Implementation
![](https://i.imgur.com/GYsq1Ap.png)
## Directly copy the following block and paste it on GradeScope after you finish the code


In [15]:
from torch import nn
class Residual_Network(nn.Module):
    def __init__(self):
        super(Residual_Network, self).__init__()
        
        self.cnn_layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
        )

        self.cnn_layer2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
        )

        self.cnn_layer3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128),
        )

        self.cnn_layer4 = nn.Sequential(
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
        )
        self.cnn_layer5 = nn.Sequential(
            nn.Conv2d(128, 256, 3, 2, 1),
            nn.BatchNorm2d(256),
        )
        self.cnn_layer6 = nn.Sequential(
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(256* 32* 32, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x1 = self.cnn_layer1(x)
        
        x1 = self.relu(x1)
        
        x2 = self.cnn_layer2(x1)
        
        x2 = self.relu(x2)
        
        x3 = self.cnn_layer3(x2)
        
        x3 = self.relu(x3)
        
        x4 = self.cnn_layer4(x3)
        
        x4 = self.relu(x4)
        
        x5 = self.cnn_layer5(x4)
        
        x5 = self.relu(x5)
        
        x6 = self.cnn_layer6(x5)
        
        x6 = self.relu(x6)
        
        # The extracted feature map must be flatten before going to fully-connected layers.
        xout = x6.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        xout = self.fc_layer(xout)
        return xout